In [3]:
import matplotlib.pyplot as plt # for plotting maps
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from pandas import read_csv

**Load datasets**

In [4]:
#Load VTD shapefile
#NOTE: This has all the relevant data from the MGGG file except for '12 Presidential & '16 Election results, demographic data

vtds=gp.read_file("./raw-from-files/VTD _shapefile/R2016_Assignment_Layers.gdb")

#Load Block-VTD Key File to Aggregate Demographic Data to the VTD Level

block_keys=pd.read_csv("./raw-from-files/block_keys/Block_Level_GeoKeys.tab",sep="\t",engine='python')

#2012 Election Data 

election_data_2012=pd.read_csv("./raw-from-files/election_data/2012 _election/results_pct_20121106.txt",delimiter=",")

#2016 Election Data 

election_data_2016=pd.read_csv("./raw-from-files/election_data/2016_election/results_pct_20161108!.txt", sep="\t",engine='python')

#MGGG Assigns districts from 2011, 2016, and new judge enacted plan 
#Taken from NCLEG Congressional Districts Shapefile

newplan= gp.read_file('./raw-from-files/district_plan/newplan/shp (3)/2016_Contingent_Congressional_Plan_Corrected.shp')

oldplan=gp.read_file('./raw-from-files/district_plan/oldplan/shp (3)/Rucho_Lewis_Congress_3.shp')

judge=gp.read_file('./raw-from-files/district_plan/judge/HB1029 3rd Edition - Shapefile/C-Goodwin-A-1-TC.shp')

#MGGG's File (Loading now to perform a quick check that the vtd shapefile is the correct one)

final_mggg=gp.read_file('./raw-from-files/mggg_file/NC_VTD/NC_VTD.shp')


In [5]:
print(vtds.columns)

Index(['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Code',
       'VTD_Name', 'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_AG_D', 'EL08G_AG_R',
       'EL08G_AG_TOT', 'EL08G_AD_D', 'EL08G_AD_R', 'EL08G_AD_TOT',
       'EL08G_CA_D', 'EL08G_CA_R', 'EL08G_CA_TOT', 'EL08G_CI_D', 'EL08G_CI_R',
       'EL08G_CI_L', 'EL08G_CI_W', 'EL08G_CI_TOT', 'EL08G_CL_D', 'EL08G_CL_R',
       'EL08G_CL_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L',
       'EL08G_GV_TOT', 'EL08G_LG_D', 'EL08G_LG_R', 'EL08G_LG_L',
       'EL08G_LG_TOT', 'EL08G_SPI_D', 'EL08G_SPI_R', 'EL08G_SPI_TOT',
       'EL08G_USS_D', 'EL08G_USS_R', 'EL08G_USS_L', 'EL08G_USS_W',
       'EL08G_USS_TOT', 'EL10G_USS_D', 'EL10G_USS_R', 'EL10G_USS_L',
       'EL10G_USS_W', 'EL10G_USS_TOT', 'EL12G_GV_D', 'EL12G_GV_R',
       'EL12G_GV_L', 'EL12G_GV_W1', 'EL12G_GV_W2', 'EL12G_GV_TOT',
       'EL12G_LG_D', 'EL12G_LG_R', 'EL12G_LG_TOT', 'EL12G_AD_D', 'EL12G_AD_R',
       'EL12G_AD_TOT', 'EL12G_CA_D', 'EL12G_CA_R', 'EL12G_CA_TOT',
       'EL12G_

**Check to Make Sure Shapefile is the Right One**

In [11]:
final_mggg["geometry"]=final_mggg.buffer(0)
vtds["geometry"]=vtds.buffer(0)
proj = vtds.crs
final_mggg = final_mggg.to_crs(proj)
file = vtds.difference(final_mggg)
print(sum(file.area)/sum(vtds.area))
print(final_mggg.geom_almost_equals(vtds,decimal=6))

**Take a Look at VTD File as well as MGGG File to see what is present / missing**

In [81]:
#print(vtds.columns)
print(final_mggg.columns)

Index(['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Name',
       'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L',
       'EL08G_GV_T', 'EL08G_USS_', 'EL08G_US_1', 'EL08G_US_2', 'EL08G_US_3',
       'EL08G_US_4', 'EL10G_USS_', 'EL10G_US_1', 'EL10G_US_2', 'EL10G_US_3',
       'EL10G_US_4', 'EL12G_GV_D', 'EL12G_GV_R', 'EL12G_GV_L', 'EL12G_GV_W',
       'EL12G_GV_1', 'EL12G_GV_T', 'EL14G_USS_', 'EL14G_US_1', 'EL14G_US_2',
       'EL14G_US_3', 'EL14G_US_4', 'Shape_Leng', 'Shape_Area', 'EL12G_PR_D',
       'EL12G_PR_R', 'EL12G_PR_L', 'EL12G_PR_W', 'EL12G_PR_1', 'EL12G_PR_T',
       'EL16G_PR_R', 'EL16G_PR_D', 'EL16G_PR_L', 'EL16G_PR_W', 'EL16G_PR_T',
       'EL16G_USS_', 'EL16G_US_1', 'EL16G_US_2', 'EL16G_US_3', 'EL16G_GV_D',
       'EL16G_GV_R', 'EL16G_GV_L', 'EL16G_GV_T', 'BPOP', 'nBPOP', 'judge',
       'newplan', 'oldplan', 'TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN',
       'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE',
       'H_BLACK

In [70]:
#pd.set_option('display.max_columns', None)
#display(vtds)
#display(final_mggg)


**Rename and Clean Columns in VTD File**

In [71]:
#Renaming the 2008 Election Data
vtds = vtds.rename(columns={"EL08G_GV_TOT": "EL08G_GV_T","EL08G_USS_D": "EL08G_USS_","EL08G_USS_R": "EL08G_US_1","EL08G_USS_L": "EL08G_US_2","EL08G_USS_W": "EL08G_US_3","EL08G_USS_TOT": "EL08G_US_4"})

#Renaming the 2010 Election Data
vtds = vtds.rename(columns={"EL10G_USS_D": "EL10G_USS_","EL10G_USS_R": "EL10G_US_1","EL10G_USS_L": "EL10G_US_2","EL10G_USS_W": "EL10G_US_3","EL10G_USS_TOT": "EL10G_US_4"})

#Renaming the 2012 Election Data Columns to match MGGG-NC
#(ASK ABOUT THIS) From merge_data: column EL12G_GV_W2 > MGGG-NC Column EL12G_GV_1 (Total number of votes for 2012 gubernatorial candidates)
#(ASK ABOUT THIS) From merge_data: column EL12G_GV_W1 > MGGG-NC Column EL12G_GV_W (Total number of votes for Donald Kreamer as 2012 gubernatorial candidates)
vtds = vtds.rename(columns={"EL12G_GV_W1": "EL12G_GV_W","EL12G_GV_W2": "EL12G_GV_1","EL12G_GV_TOT": "EL12G_GV_T"})

#Renaming the 2014 Election Data
vtds = vtds.rename(columns={"EL14G_USS_R": "EL14G_USS_","EL14G_USS_D": "EL14G_US_1","EL14G_USS_L": "EL14G_US_2","EL14G_USS_WI": "EL14G_US_3","EL14G_USS_TOT": "EL14G_US_4"})

In [72]:
print(vtds.columns)

Index(['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Name',
       'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L',
       'EL08G_GV_T', 'EL08G_USS_', 'EL08G_US_1', 'EL08G_US_2', 'EL08G_US_3',
       'EL08G_US_4', 'EL10G_USS_', 'EL10G_US_1', 'EL10G_US_2', 'EL10G_US_3',
       'EL10G_US_4', 'EL12G_GV_D', 'EL12G_GV_R', 'EL12G_GV_L', 'EL12G_GV_W',
       'EL12G_GV_1', 'EL12G_GV_T', 'EL14G_USS_', 'EL14G_US_1', 'EL14G_US_2',
       'EL14G_US_3', 'EL14G_US_4', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')


In [73]:
vtds = vtds[['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Name',
       'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L',
       'EL08G_GV_T', 'EL08G_USS_', 'EL08G_US_1', 'EL08G_US_2', 'EL08G_US_3',
       'EL08G_US_4', 'EL10G_USS_', 'EL10G_US_1', 'EL10G_US_2', 'EL10G_US_3',
       'EL10G_US_4', 'EL12G_GV_D', 'EL12G_GV_R', 'EL12G_GV_L', 'EL12G_GV_W',
       'EL12G_GV_1', 'EL12G_GV_T', 'EL14G_USS_', 'EL14G_US_1', 'EL14G_US_2',
       'EL14G_US_3', 'EL14G_US_4', 'Shape_Length', 'Shape_Area','geometry']]

Remaining work: Add in demographic data, district assignments from 3 plans and '12 presidential + '16 results.

**Load Census API**

Retrieve demographic data from Census API

To see the full list of census variables, look here: https://api.census.gov/data/2010/dec/sf1/variables.html

In [11]:
# Code chunk by Melisande Teng - VRDI 2019
variables = [
    # pop
    "P005001",
    "P005003",
    "P005004",
    "P005005",
    "P005006",
    "P005007",
    "P005008",
    "P005009",
    "P005010",
    # vap
    "P011001",
    "P011002",
    "P011005",
    "P011006",
    "P011007",
    "P011008",
    "P011009",
    "P011010",
    "P011011",
]

keys = [
    # pop
    "TOTPOP",  # these follow from above, so TOTPOP = P005001
    "NH_WHITE",
    "NH_BLACK",
    "NH_AMIN",
    "NH_ASIAN",
    "NH_NHPI",
    "NH_OTHER",
    "NH_2MORE",
    "HISP",
    # vap
    "VAP",
    "HVAP",
    "WVAP",
    "BVAP",
    "AMINVAP",
    "ASIANVAP",
    "NHPIVAP",
    "OTHERVAP",
    "2MOREVAP",
]

import requests

def counties(state_fips):
    """Inputs: state fips code
    Process: Retrieves a list of counties in the given state from the Census API.  
    Outputs: A list of county fips codes in the state. """
    resp = requests.get(
        "https://api.census.gov/data/2010/dec/sf1"
        "?get=NAME&for=county:*&in=state:{}".format(state_fips)
    )
    header, *rows = resp.json()
    county_column_index = header.index("county")
    county_fips_codes = set(row[county_column_index] for row in rows)
    return county_fips_codes


def block_data_for_county(state_fips, county_fips, variables=variables, keys=keys):
    """Inputs: state fips code, county fips code, list of variables and corresponding keys. 
    Process: formats the query to call the Census API. 
    Outputs: data for the county in a pandas dataframe. """
    url = (
        "https://api.census.gov/data/2010/dec/sf1"
        + "?get={},NAME&for=block:*".format(",".join(variables))
        + "&in=state:{}&in=county:{}&in=tract:*".format(state_fips, county_fips)
    )
    resp = requests.get(url)
    header, *rows = resp.json()
    variable_lookup = dict(zip(variables, keys))
    columns = [variable_lookup.get(column_name, column_name) for column_name in header]
    dtypes = {key: int for key in keys}
    dtypes.update({key: str for key in ["state", "county", "tract", "block"]})
    data = pd.DataFrame.from_records(rows, columns=columns).astype(dtypes)
    data["geoid"] = data["state"] + data["county"] + data["tract"] + data["block"]
    return data


def block_data_for_state(state_fips):
    """Input: state_fips
    Process: Retrieve a list of county fips codes in the state. 
        The Census API only lets you query one county at a time, so you first need the full list. 
        Then call block_data_for_county() to retrieve the data at the county level. 
    Outputs: block-level data for the state fips for the list of variables defined above. 
    """
    from tqdm.auto import tqdm
    county_fips_codes = counties(state_fips)
    return pd.concat(
        [
            block_data_for_county(state_fips, county_fips)
            for county_fips in tqdm(county_fips_codes)
        ]
    )

In [12]:
#if running script for the first time, use this chunk
nc = block_data_for_state('37')
nc = nc.rename(columns={"geoid": "GEOID10"})
nc.to_csv('nc_census_2010_blocks2.csv')

Example of how you would write the census data to a csv and then re-load it so you don't have to query it every time you re-run this script


In [13]:
 #nc = pd.read_csv('./nc_census_2010_blocks2.csv')
 #nc = nc.astype({"GEOID10": str})

Take a look at the census data

In [43]:
nc.head(2)

,TOTPOP,NH_WHITE,NH_BLACK,NH_AMIN,NH_ASIAN,NH_NHPI,NH_OTHER,NH_2MORE,HISP,VAP,...,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,NAME,state,county,tract,block,Block_Key
0,7,0,7,0,0,0,0,0,0,7,...,0,0,0,0,"Block 1005, Block Group 1, Census Tract 202, E...",37,065,020200,1005,370650202001005
1,40,0,39,1,0,0,0,0,0,25,...,0,0,0,0,"Block 1006, Block Group 1, Census Tract 202, E...",37,065,020200,1006,370650202001006


**Prorate Demographic and Voting Population Data from the Block Level to VTD using Block_Keys**

Using Block Keys because that was the method MGGG specified. 

In [44]:
print(block_keys.shape)
print(nc.shape)
nc = nc.rename(columns={"GEOID10": "Block_Key"})
nc.head(5)
block_keys.head(5)

(288987, 17)
(288987, 24)


,Block_Key,Block_Code,BG_Key,BG_Code,Tract_Key,Tract_Code,Tract_Name,VTD_Key,VTD_Code,VTD_Name,Cnty_Key,Cnty_Code,Cnty_Name,Place_Key,Place_Code,Place_Name,ZCTA5
0,370010201001000,1000,370010201001,1,37001020100,20100,201.0,3700112W,12W,12W,37001,1,Alamance,3709060.0,9060.0,Burlington,27215.0
1,370010201001001,1001,370010201001,1,37001020100,20100,201.0,3700112W,12W,12W,37001,1,Alamance,3709060.0,9060.0,Burlington,27215.0
2,370010201001002,1002,370010201001,1,37001020100,20100,201.0,3700112W,12W,12W,37001,1,Alamance,3709060.0,9060.0,Burlington,27215.0
3,370010201001003,1003,370010201001,1,37001020100,20100,201.0,3700112W,12W,12W,37001,1,Alamance,3709060.0,9060.0,Burlington,27217.0
4,370010201001004,1004,370010201001,1,37001020100,20100,201.0,3700112W,12W,12W,37001,1,Alamance,3709060.0,9060.0,Burlington,27215.0


Convert both "Block_Key" columns to integers so they can be joined

In [45]:
nc["Block_Key"]=nc["Block_Key"].astype(int)
block_keys["Block_Key"]=block_keys["Block_Key"].astype(int)

In [46]:
block_joined=pd.merge(nc,block_keys,on='Block_Key',how='outer',indicator=True)

In [47]:
print(block_joined.head(10))
print(block_joined.shape)

   TOTPOP  NH_WHITE  NH_BLACK  NH_AMIN  NH_ASIAN  NH_NHPI  NH_OTHER  NH_2MORE  \
0       7         0         7        0         0        0         0         0   
1      40         0        39        1         0        0         0         0   
2      30         0        30        0         0        0         0         0   
3      31         2        26        0         0        0         0         0   
4      35         4        28        1         0        0         0         2   
5      20         1        19        0         0        0         0         0   
6       4         1         3        0         0        0         0         0   
7      12         0        12        0         0        0         0         0   
8      30         0        30        0         0        0         0         0   
9      24         2        21        0         0        0         0         0   

   HISP  VAP  ...  VTD_Code  VTD_Name  Cnty_Key  Cnty_Code  Cnty_Name  \
0     0    7  ...      1202      12

Both dataframes originally had 288,987 rows so join looks good

In [48]:
#list(block_joined.columns)

**Going to pivot this dataframe by the "VTD_Key" column

**Pivot Block Level Census Data to VTD Level by VTD Key**

In [49]:
census_vtd=pd.pivot_table(block_joined,index="VTD_Key",aggfunc=sum)
census_vtd.reset_index(inplace=True)

**Check to make sure that VTD row number makes sense**

In [50]:
print(census_vtd.shape)
print(vtds.shape)

(2692, 31)
(2692, 85)


**Merge Census VTD Data with the VTD Shapefile by "VTD Key"**

In [51]:
vtds_joined = pd.merge(census_vtd,vtds,on='VTD_Key',how='outer',indicator=True)

**Check to make sure that joined VTD row number makes sense**

In [52]:
print(vtds_joined.shape)

(2692, 116)


**Next Step: Tie VTDs to their District Assignment Across the 3 Plans**

MGGG Assigns districts from 2011, 2016, and new judge enacted plan 
Taken from NCLEG Congressional Districts Shapefile

"newplan" is the 2016 plan  
"oldplan" is the 2011 plan  
"judge" is the new judge plan  

**Set Shapefile Projections**

In [76]:
vtds_joined = gp.GeoDataFrame(vtds_joined, geometry='geometry')
vtds_joined["geometry"]=vtds_joined.buffer(0)
proj=vtds_joined.crs

***2016 plan - "newplan"***

In [85]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
#print(newplan)
newplan=newplan.to_crs(proj)
newplan["geometry"] = newplan.buffer(0)
assignments=maup.assign(vtds_joined,newplan)
assignments=assignments+1
#There are 13 Congressional Districts in North Carolina, but there are no 0 districts, 
#We have to add 1.0 to each districts because Python indexing. 
vtds_joined["newplan"]=assignments

**2011 plan - "oldplan"**

In [86]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
#print(oldplan)
oldplan=oldplan.to_crs(proj)
oldplan["geometry"] = oldplan.buffer(0)
assignments=maup.assign(vtds_joined,oldplan)
assignments=assignments+1
#There are 13 Congressional Districts in North Carolina, but there are no 0 districts, 
#We have to add 1.0 to each districts because Python indexing. 
vtds_joined["oldplan"]=assignments

**Judge plan -"judge"**

In [88]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
#print(oldplan)
judge=judge.to_crs(proj)
judge["geometry"] = judge.buffer(0)
assignments=maup.assign(vtds_joined,judge)
assignments=assignments+1
#There are 13 Congressional Districts in North Carolina, but there are no 0 districts, 
#We have to add 1.0 to each districts because Python indexing. 
vtds_joined["judge"]=assignments

**Next Step: Joining '12 Presidential and '16 Election Data**

In [2]:
election_data_2012.head(2)
election_data_2016.head(2)

NameError: name 'election_data_2012' is not defined

In [1]:
#create dictionary of state + county fips

my_county= {
    'ALAMANCE':'37001', 'ALEXANDER':'37003','ANSON':'37007','ASHE':'37009', 'AVERY':'37011', 'BEAUFORT':'37013', 'BERTIE':'37015', 'BLADEN':'37017',  'BRUNSWICK':'37019',
    'BUNCOMBE':'37021','BURKE':'37023','CABARRUS':'37025','CALDWELL':'37027','CAMDEN':'37029','CARTERET':'37031','CASWELL':'37033','CATAWBA':'37035','CHATHAM':'37037','CHEROKEE':'37039','CHOWAN':'37041','CLAY':'37043','CLEVELAND':'37045',
    'COLUMBUS':'37047','CRAVEN':'37049','CUMBERLAND':'37051','CURRITICK':'37053','DARE':'37055','DAVIDSON':'37057','DAVIE':'37059','DUPLIN':'37061','DURHAM':'37063','EDGECOMBE':'37065',
    'FORSYTH':'37067','FRANKLIN':'37069','GASTON':'37071','GATES':'37073','GRAHAM':'37075','GRANVILLE':'37077',
    'GREENE':'37079','GUILFORD':'37081','HALIFAX':'37083','HARNETT':'37085','HAYWOOD':'37087','HENDERSON':'37089','HERTFORD':'37091','HOKE':'37093','HYDE':'37095',
    'IREDELL':'37097','JACKSON':'37099','JOHNSTON':'37101','JONES':'37103','LEE':'37105','LENOIR':'37107','LINCOLN':'37109','MCDOWELL':'37111','MACON':'37113','MADISON':'37115',
    'MERTIN':'37117','MECKLENBURG':'37119','MITCHELL':'37121','MONTGOMERY':'37123','MOORE':'37125','NASH':'37127','NEW HANOVER':'37129','NORTHAMPTON':'37131','ONSLOW':'37133','ORANGE':'37135','PAMLICO':'37137','PASQUOTANK':'37139','PENDER':'37141','PERQUIMANS':'37143','PERSON':'37145','PITT':'37147','POLK':'37149','RANDOLPH':'37151',
    'RICHMOND':'37153','ROCKINGHAM':'37157','ROWAN':'37159','RUTHERFORD':'37161','SAMPSON':'37163','SCOTLAND':'37165','STANLY':'37167','STOKES':'37169','SURRY':'37171','SWAIN':'37173','TRANSYLVANIA':'37175','TYRRELL':'37177','UNION':'37179','VANCE':'37181','WAKE':'37183','WARREN':'37185','WASHINGTON':'37187','WATAUGA':'37189','WAYNE':'37191','WILKES':'37193','WILSON':'37195','YADKIN':'37197',  'YANCEY':'37199'
}

*Election Data 2012*

In [215]:
election_data_2012['countyID']=election_data_2012['county'].map(my_county)
election_data_2016['countyID']=election_data_2012['county'].map(my_county)

#Preparation to string together VTD_Key
election_data_2012['countyID']=election_data_2012['countyID'].astype(str)
election_data_2012['precinct']=election_data_2012['precinct'].astype(str)
election_data_2012['precinctID']=election_data_2012['precinct'].str.zfill(2)
election_data_2012["precinctID"]=election_data_2012["precinct"].str.split("_", n=1, expand=True) 

#Combining state id, county fips, and precinct id to make VTD_Key
election_data_2012['VTD_Key']=election_data_2012['countyID']+election_data_2012['precinctID']

#Preparation to string together VTD_Key
election_data_2016['countyID']=election_data_2016['countyID'].astype(str)
election_data_2016['Precinct']=election_data_2016['Precinct'].astype(str)
election_data_2016['precinctID']=election_data_2016['Precinct'].str.zfill(2)

#Combining state id, county fips, and precinct id to make VTD_Key
election_data_2016['VTD_Key']=election_data_2016['countyID']+election_data_2016['precinctID']

In [216]:
#Removing rows that has 'ONE STOP', 'ABSENTEE BY MAIL','PROVISIONAL', 'TRANSFER','CURBSIDE', 'ACCUMULATED'
#create a dataframe that has the above words

searchfor=['ABSENTEE','PROVISIONAL','TRANSFER']
#nan=['nan']
#remove rows that contains words from searchfor dataframe

election_data_2012=election_data_2012[~election_data_2012.precinct.str.contains('|'.join(searchfor))]
#election_data_2012=election_data_2012[~election_data_2012.county.str.contains('|'.join(nan))]

#Filling in empty columns in party column to write in 
#election_data_2012.loc[election_data_2012["party"].isnull(),'party']="Write_In"

In [217]:
election_data_2012.head()

,county,precinct,contest_type,runoff_status,recount_status,contest,choice,winner_status,party,Election Day,One Stop,Absentee by Mail,Provisional,total votes,district,countyID,precinctID,VTD_Key
0,ALAMANCE,01_PATTERSON,S,0,0,PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES,Obama/Biden,0,DEM,225,237,21,5,488,Not Found,37001,01,3700101
1,ALAMANCE,02_COBLE,S,0,0,PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES,Obama/Biden,0,DEM,219,184,26,0,429,Not Found,37001,02,3700102
2,ALAMANCE,035_BOONE 5,S,0,0,PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES,Obama/Biden,0,DEM,409,362,52,9,832,Not Found,37001,035,37001035
3,ALAMANCE,03C_CENTRAL BOONE,S,0,0,PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES,Obama/Biden,0,DEM,203,288,35,0,526,Not Found,37001,03C,3700103C
4,ALAMANCE,03N_NORTH BOONE,S,0,0,PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES,Obama/Biden,0,DEM,380,429,70,2,881,Not Found,37001,03N,3700103N


In [218]:
office_list_2012=["PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES"]
party_list=["REP","DEM","LIB","Write_In"]
election_data_2012=election_data_2012[election_data_2012['contest'].isin(office_list_2012)]
election_data_2012=election_data_2012[election_data_2012['party'].isin(party_list)]

In [219]:
#pivoting the election 2012 dataset
election_data_2012_p=pd.pivot_table(election_data_2012,index="VTD_Key",columns=["party"],values='total votes',aggfunc=sum)
election_data_2012_p.columns=["EL12G_PR_D","EL12G_PR_L","EL12G_PR_R"]
election_data_2012_p.reset_index(inplace=True)


In [220]:
election_data_2012_p.head(2)

,VTD_Key,EL12G_PR_D,EL12G_PR_L,EL12G_PR_R
0,3700101,488,16,1661
1,3700102,429,32,1936


In [221]:
election_data_2012_p.shape

(2845, 4)

In [222]:
contain_values=election_data_2012[election_data_2012['VTD_Key'].str.contains('3715920')&election_data_2012['total votes']==None]
print(contain_values)

Empty DataFrame
Columns: [county, precinct, contest_type, runoff_status , recount_status, contest, choice, winner_status, party, Election Day, One Stop, Absentee by Mail, Provisional, total votes, district, countyID, precinctID, VTD_Key]
Index: []


*Election Data 2016*

In [225]:
#Removing rows that has 'ONE STOP', 'ABSENTEE BY MAIL','PROVISIONAL', 'TRANSFER','CURBSIDE', 'ACCUMULATED'
#create a dataframe that has the above words

#searchfor=['ABSENTEE','PROVISIONAL','TRANSFER']
#nan=['nan']
#remove rows that contains words from searchfor dataframe

#election_data_2016=election_data_2016[~election_data_2016.Precinct.str.contains('|'.join(searchfor))]
#election_data_2016=election_data_2016[~election_data_2016.County.str.contains('|'.join(nan))]

In [226]:
#filtering the dataset

office_list_2016=["US PRESIDENT","US SENATE","NC GOVERNOR"]
party_list=["REP","DEM","LIB"]
election_data_2016=election_data_2016[election_data_2016['Contest Name'].isin(office_list_2016)]
election_data_2016=election_data_2016[election_data_2016['Choice Party'].isin(party_list)]
#election_2016=election_2016_filter[election_2016_filter['Choice'].isin(write_in)]


In [227]:
#pivoting the election 2016 dataset
election_data_2016_p=pd.pivot_table(election_data_2016,index="VTD_Key",columns=["Contest Name","Choice Party"],values='Total Votes',aggfunc=sum)
election_data_2016_p.columns=["EL16G_GV_D","EL16G_GV_L","EL16G_GV_R","EL16G_PR_D","EL16G_PR_L","EL16G_PR_R","EL16G_US_1","EL16G_US_2","EL16G_USS_"]
election_data_2016_p.reset_index(inplace=True)
election_data_2016_p.head(2)


,VTD_Key,EL16G_GV_D,EL16G_GV_L,EL16G_GV_R,EL16G_PR_D,EL16G_PR_L,EL16G_PR_R,EL16G_US_1,EL16G_US_2,EL16G_USS_
0,3700101,499,45,1806,411,50,1865,418,89,1829
1,3700102,497,39,1927,403,40,2004,396,79,1973


In [228]:
contain_values=election_data_2012[election_data_2012['VTD_Key'].str.contains('3715920')]
print(contain_values)


Empty DataFrame
Columns: [county, precinct, contest_type, runoff_status , recount_status, contest, choice, winner_status, party, Election Day, One Stop, Absentee by Mail, Provisional, total votes, district, countyID, precinctID, VTD_Key]
Index: []


In [320]:
election_data=pd.merge(vtd_election_data,election_data_2012_p,on='VTD_Key',how='outer',indicator='EL12')
election_data=pd.merge(election_data,election_data_2016_p,on='VTD_Key',how='outer',indicator='EL16')


In [322]:
election_data.shape

(4182, 92)

In [234]:
election_data.to_csv('election_data_joined.csv')

**Look at MGGG FINAL FILE (loaded above)**

In [328]:
final_mggg.head(2)

,ALAND10,AWATER10,VTD,County,VTD_Key,VTD_Name,PL10AA_TOT,PL10VA_TOT,EL08G_GV_D,EL08G_GV_R,...,VAP,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,geometry
0,1951716.0,32157.0,3700106W,37001,3700106W,06W,1973,1505,330,301,...,1505,211,899,338,4,32,0,0,21,"POLYGON ((572318.656 259054.912, 572364.420 25..."
1,2887286.0,0.0,3700112E,37001,3700112E,12E,3391,2503,586,263,...,2503,622,1085,712,19,38,0,2,25,"POLYGON ((572318.656 259054.912, 572310.834 25..."


In [329]:
final_mggg.shape

(2692, 85)

In [330]:
merge_data.columns.unique()

Index(['VTD_Key', '2MOREVAP', 'ALAND10_x', 'AMINVAP', 'ASIANVAP', 'AWATER10_x',
       'BG_Code', 'BG_Key', 'BVAP', 'Block_Code',
       ...
       'EL16G_GV_L', 'EL16G_GV_R', 'EL16G_PR_D', 'EL16G_PR_L', 'EL16G_PR_R',
       'EL16G_US_1', 'EL16G_US_2', 'EL16G_USS_', 'EL16', 'merge_data'],
      dtype='object', length=148)

**Cleaning the merge_data file so that it is consistent with MGGG NC**

In [335]:
## Choose columns from merge_data that is also present in the final mggg columns
final_merge=merge_data[['ALAND10_y','AWATER10_y','VTD_Key','VTD_Name','PL10AA_TOT','PL10VA_TOT','EL08G_GV_D','EL08G_GV_R','EL08G_GV_L','EL08G_GV_T','EL08G_USS_','EL08G_US_1','EL08G_US_2','EL08G_US_3','EL08G_US_4','EL10G_USS_','EL10G_US_1','EL10G_US_2','EL10G_US_3','EL12G_GV_D','EL14G_USS_','EL14G_US_1','EL14G_US_2','EL14G_US_3','EL14G_US_4','TOTPOP','NH_WHITE','NH_BLACK','NH_AMIN','NH_ASIAN','NH_NHPI','NH_OTHER','NH_2MORE','HISP','VAP','HVAP','WVAP','BVAP','AMINVAP','ASIANVAP','NHPIVAP','OTHERVAP','2MOREVAP','geometry','EL12G_PR_D','EL12G_PR_R','EL12G_PR_L','EL16G_GV_D','EL16G_GV_L','EL16G_GV_R','EL16G_PR_D','EL16G_PR_L','EL16G_PR_R','EL16G_US_1','EL16G_US_2','EL16G_USS_','oldplan','newplan','judge']]

In [336]:
#final_merge=merge_data[['VTD_Key','PL10AA_TOT','PL10VA_TOT','EL08G_GV_D','EL08G_GV_R','EL08G_GV_L','EL08G_GV_T','EL08G_USS_','EL08G_US_1','EL08G_US_2','EL08G_US_3','EL08G_US_4','EL10G_USS_','EL10G_US_1','EL10G_US_2','EL10G_US_3','EL12G_GV_D','EL14G_USS_','EL14G_US_1','EL14G_US_2','EL14G_US_3','EL14G_US_4','TOTPOP','NH_WHITE','NH_BLACK','NH_AMIN','NH_ASIAN','NH_NHPI','NH_OTHER','NH_2MORE','HISP','VAP','HVAP','WVAP','BVAP','AMINVAP','ASIANVAP','NHPIVAP','OTHERVAP','2MOREVAP','EL12G_PR_D','EL12G_PR_R','EL12G_PR_L','EL16G_GV_D','EL16G_GV_L','EL16G_GV_R','EL16G_PR_D','EL16G_PR_L','EL16G_PR_R','EL16G_US_1','EL16G_US_2','EL16G_USS_']]

In [337]:
final_merge.shape

(4182, 59)

**Validation**

In [338]:
validate=pd.merge(final_merge,final_mggg, on=['VTD_Key'],how='inner')


In [339]:
#rows before and after are the same
validate.shape

(2692, 143)

In [340]:
final_mggg.shape

(2692, 85)

**Check population**

In [341]:
validate['pop_diff']=validate.TOTPOP_x-validate.TOTPOP_y
validate[validate.pop_diff.abs()<10].shape[0]/validate.shape[0]

1.0

**Check Voting age population**

In [342]:
validate['bvap_diff']=validate.BVAP_x-validate.BVAP_y
validate[validate.bvap_diff.abs()<10].shape[0]/validate.shape[0]

1.0

**Check 08 election data**

In [343]:
validate['election_08_diff']=validate.EL08G_GV_D_x-validate.EL08G_GV_D_y
validate[validate.election_08_diff.abs()<10].shape[0]/validate.shape[0]

1.0

**Check 12 election data**

In [344]:
#validate['election_12_diff']=validate.EL12G_PR_D_x-validate.EL12G_PR_D_y
#validate[validate.election_12_diff.abs()<10].shape[0]/validate.shape[0]

In [345]:
validate[['VTD_Key','EL12G_PR_D_x','EL12G_PR_D_y']].head(10)

,VTD_Key,EL12G_PR_D_x,EL12G_PR_D_y
0,3700101,488.0,488
1,3700102,429.0,429
2,37001035,832.0,832
3,3700103C,526.0,526
4,3700103N,881.0,1143
5,3700103S,1380.0,1380
6,3700103W,568.0,568
7,3700104,627.0,627
8,3700105,365.0,365
9,37001063,987.0,987


#Election Data Join Looks Good

**Check 16 election data**

In [346]:
validate[['VTD','EL16G_PR_R_x','EL16G_PR_R_y']].head(10)

,VTD,EL16G_PR_R_x,EL16G_PR_R_y
0,3700101,1865.0,1865
1,3700102,2004.0,2004
2,37001035,NaN,1485
3,3700103C,1059.0,1059
4,3700103N,1160.0,1374
5,3700103S,2045.0,2045
6,3700103W,1201.0,1201
7,3700104,2043.0,2043
8,3700105,1448.0,1448
9,37001063,NaN,528


In [347]:
validate[['VTD','EL16G_US_2_x','EL16G_US_2_y']].head(10)

,VTD,EL16G_US_2_x,EL16G_US_2_y
0,3700101,89.0,89
1,3700102,79.0,79
2,37001035,NaN,96
3,3700103C,57.0,57
4,3700103N,81.0,86
5,3700103S,115.0,115
6,3700103W,72.0,72
7,3700104,88.0,88
8,3700105,72.0,72
9,37001063,NaN,57


Getting sum of election results by party from MGGG file

In [348]:
#total votes for presidential republican party based on MGGG file
total_r=final_mggg['EL16G_PR_R'].sum()
print(total_r)

2359749


In [349]:
total_d=final_mggg['EL16G_PR_D'].sum()
print(total_d)

2180316


In [350]:
total_l=final_mggg['EL16G_PR_L'].sum()
print(total_l)

129929


Getting sum of election results by party for election data as obtained from state board of elections

In [351]:

total_r_p=election_data['EL16G_PR_R'].sum()
print(total_r_p)

2362631.0


In [352]:

total_d_p=election_data['EL16G_PR_D'].sum()
print(total_d_p)

2189316.0


In [359]:
total_l_p=election_data['EL16G_PR_L'].sum()
print(total_l_p)

130126.0


Total Election Votes from 2016 from MGGG file does not match up with the data available from the state board of elections (validated on the website and from the State Board of Election datafil)

**Validate Enacted Plans**

In [354]:
validate['newplan']=validate.newplan_x-validate.newplan_y
validate[validate.newplan.abs()<10].shape[0]/validate.shape[0]

1.0

In [355]:
validate['judge']=validate.judge_x-validate.judge_y
validate[validate.judge.abs()<10].shape[0]/validate.shape[0]

0.8763001485884101

In [356]:
validate['oldplan']=validate.oldplan_x-validate.oldplan_y
validate[validate.oldplan.abs()<10].shape[0]/validate.shape[0]

0.0

In [357]:
validate[['oldplan_x','oldplan_y']].head(10)

,oldplan_x,oldplan_y
0,6.0,296.0
1,10.0,297.0
2,10.0,297.0
3,10.0,297.0
4,10.0,297.0
5,10.0,297.0
6,10.0,297.0
7,10.0,297.0
8,10.0,297.0
9,8.0,300.0


In [358]:
validate.oldplan_y.unique()

array([296., 297., 300., 301., 305., 303., 298., 294., 295., 299., 306.,
       302., 304.])

MGGG's dataset was not reseted, there are 13 congressional districts here